In [4]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
sx = qt.sigmax();sy = qt.sigmay(); sz = qt.sigmaz(); s0 = qt.qeye(2)

In [156]:

def update_xi(xi, dt, tc, sig):
    #Will be similair function for other models as well
    n = np.random.normal(0,1)
    xi = xi*np.exp(-dt/tc)+np.sqrt(1-np.exp(-2*dt/tc))*sig*n
    #print("post"+str(self.xi))
    return xi

hbar = 0.65821195


def measure(basisA, basisB, psi):
    op_pp = qt.tensor((s0+basisA)/2,(s0+basisB)/2)
    op_mm = qt.tensor((s0-basisA)/2,(s0-basisB)/2)
    op_pm = qt.tensor((s0+basisA)/2,(s0-basisB)/2)
    op_mp = qt.tensor((s0-basisA)/2,(s0+basisB)/2)
    prob_pp = (qt.expect(op_pp,psi))
    prob_mm = (qt.expect(op_mm,psi))
    prob_pm = (qt.expect(op_pm,psi))
    prob_mp = (qt.expect(op_mp,psi))
    #print(prob_pp+prob_mm+prob_pm+prob_mp)
    draw = np.random.uniform()
    if draw < prob_pp:
        return (1,1)
    elif draw < prob_pp+prob_mm:
        return (0,0)
    elif draw < prob_pp+prob_mm+prob_pm:
        return (1,0)
    else:
        return (0,1)

    
def get_results(basis, register):
    results = []
    for r in register:
        if r[2]==basis:
            results.append([r[0], r[1], r[3][0],r[3][1]])
    return np.array(results)
    

In [157]:
def get_results(sig, tc, dt, th, trials):
    basisA = [sx*np.cos(th)+sz*np.sin(th),sz*np.cos(th)-sx*np.sin(th)]
    basisB = [sx,sz]
    psi = qt.ghz_state(2)
    results = []
    xis = [0]
    for tr in range(trials):
        A_m = np.random.randint(len(basisA))
        B_m = np.random.randint(len(basisB))
        shot = measure(basisA[A_m],basisB[B_m],psi)
        psi = (1j*qt.tensor(sx,sz)*xis[-1]).expm()*qt.ghz_state(2)
        #print(qt.expect(qt.tensor(basisA[A_m],basisB[B_m]),psi))
        results.append([tr, xis[-1], (A_m, B_m), (shot[0],shot[1])])
        xis.append(update_xi(xis[-1], dt, tc, sig))
    return results

#2323

In [187]:
def get_joint_probability(register):
    bases = list(set([r[2] for r in register]))
    bases.sort()
    probs = np.zeros((len(register),len(bases),4))
    for rn,r in enumerate(register):
        if rn>0:
            probs[rn:, bases.index(r[2]),2*r[3][0]+r[3][1]] += 1
        else:
            probs[rn:, bases.index(r[2]),2*r[3][0]+r[3][1]] = 1
        probs[rn] = probs[rn]/(rn+1)
        
    return probs
        
    

In [188]:
tc = 5
sig = 0.0
th = 1
trials = 50
res = get_results(sig, tc, dt, th, trials)
probs = get_joint_probability(res)

In [193]:
probs[-1]

array([[0.  , 0.02, 0.  , 0.12],
       [0.1 , 0.02, 0.02, 0.08],
       [0.  , 0.26, 0.14, 0.  ],
       [0.12, 0.06, 0.02, 0.04]])

In [ ]:
for prob in probs:
    for k in range(4):
        plt.plot()